In [1]:
import numpy as np
import cv2
import random
import pandas as pd


import matplotlib.pyplot as plt
from seaborn import load_dataset

In [2]:
imagenReferencia = cv2.imread('/home/miguel/Descargas/Proyecto Tesis/Imagenes Originales Gris/Baboon_gray.png')
imagenReferenciaGris = cv2.cvtColor(imagenReferencia, cv2.COLOR_RGB2GRAY)
imagenRuido = cv2.imread('/home/miguel/Descargas/Proyecto Tesis/Baboon/Baboon_gray_512_512_variance_225.png')
imagenRuidoGris = cv2.cvtColor(imagenRuido, cv2.COLOR_RGB2GRAY)
areaBusqueda = 21
tamañoParche = 8

# Algoritmo de búsqueda exhaustiva

In [4]:
# imgAct: Imagen actual puede ser ruidosa (imagen de la cual se adquiere la plantilla)
# areaBus: Área de búsqueda 
# tamParche: tamaño del parche (parche dentro del área de búsqueda)

def algoritmoBusquedaExhaustiva(posicionAltoImagenActual,posicionAnchoImagenActual,imgAct,imgRuido,areaBus,tamParche):   
    
    imagenActual = imgAct[posicionAltoImagenActual:posicionAltoImagenActual+tamParche, posicionAnchoImagenActual: posicionAnchoImagenActual+tamParche]
    
    dh1 = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for m in range(posicionAltoImagenActual,posicionAltoImagenActual+areaBus-tamParche+1):
        for n in range(posicionAnchoImagenActual,posicionAnchoImagenActual+areaBus-tamParche+1):
            imagenReferencia = imgRuido[m:m+tamParche, n: n+tamParche]
            ssd=0
            for rx_Pache in range (0,tamParche):
                for ry_Pache in range (0,tamParche):
                    resta = int(imagenActual[rx_Pache,ry_Pache])-int(imagenReferencia[rx_Pache,ry_Pache])
                    ssd += resta**2
#                     ssd += np.sum(np.power((imagenActual[rx_Pache,ry_Pache]-imagenReferencia[rx_Pache,ry_Pache]),2))
            dh1 = dh1.append({'ssd': ssd, 'coordenada_x':m, 'coordenada_y':n}, ignore_index=True)
#     print(posicionAltoImagenActual,posicionAnchoImagenActual)
    by_ssd = dh1.sort_values('ssd',ascending=True)
    by_ssd = by_ssd.reset_index(drop=True)
#     by_ssd.head()
    posicionAltoImagenReferencia = by_ssd['coordenada_x'][0]
    posicionAnchoImagenReferencia = by_ssd['coordenada_y'][0]
    
#     print(by_ssd)
    return posicionAltoImagenReferencia,posicionAnchoImagenReferencia
    
        
    

In [5]:
posicionAltoImagenReferencia,posicionAnchoImagenReferencia = algoritmoBusquedaExhaustiva(210,210,imagenReferenciaGris,imagenRuidoGris,areaBusqueda,tamañoParche)

# plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
# plt.show()
# print(imagen)
# cv2.imwrite('8x8.png', imagen)

In [9]:
pxy_pref_ABE = pd.DataFrame(columns=['coordenada_x', 'coordenada_y','referencia_x','referencia_y',])
altoImagen = imagenReferenciaGris.shape[0]
anchoImagen = imagenReferenciaGris.shape[1]

for posicionAlto in range(0,altoImagen-areaBusqueda): # altoImagen-areaBusqueda
    for posicionAncho in range(0,anchoImagen-areaBusqueda): # anchoImagen-areaBusqueda
        posicionAltoImagenReferencia,posicionAnchoImagenReferencia = algoritmoBusquedaExhaustiva(posicionAlto,posicionAncho,imagenReferenciaGris,imagenRuidoGris,areaBusqueda,tamañoParche)
        pxy_pref_ABE = pxy_pref_ABE.append({'coordenada_x': posicionAlto, 'coordenada_y':posicionAncho, 'referencia_x':posicionAltoImagenReferencia,'referencia_y':posicionAnchoImagenReferencia}, ignore_index=True)
    
    
        
pxy_pref_ABE

,coordenada_x,coordenada_y,referencia_x,referencia_y
0,0,0,0,0
1,0,1,0,1
2,0,2,0,2
3,0,3,0,3
4,0,4,0,4
...,...,...,...,...
241076,490,486,499,498
241077,490,487,500,498
241078,490,488,500,498
241079,490,489,500,498


In [10]:
suma = 0
miu = 0
for i in range(0,len(pxy_pref_ABE)-1):
    if pxy_pref_ABE['coordenada_x'][i] == pxy_pref_ABE['referencia_x'][i] and pxy_pref_ABE['coordenada_y'][i] == pxy_pref_ABE['referencia_y'][i]:
        suma += 1

miu =  suma/(len(pxy_pref_ABE)-1)
miu
    

0.5542724406835905

Sin filtro
# Efectivo/lena_gray_512_512_variance_25_Efectivo
0.41876970300315247
# Efectivo/lena_gray_512_512_variance_100_Efectivo.png
0.3920565787290526
# Efectivo/lena_gray_512_512_variance_225_Efectivo.png
0.35327692052430726
# Efectivo/lena_gray_512_512_variance_400_Efectivo.png
0.31881533101045295

# Baboon/Baboon_gray_512_512_variance_25.png:
0.91674962667994

# Baboon/Baboon_gray_512_512_variance_100.png

# Baboon/Baboon_gray_512_512_variance_225.png

# Baboon/Baboon_gray_512_512_variance_400.png
0.409610917537747


# lena/lena_gray_512_512_variance_25.png:
0.32918118466898955
# lena/lena_gray_512_512_variance_100.png:
0.22643106022896964
# lena/lena_gray_512_512_variance_225.png:
0.15557076489132238
# lena/lena_gray_512_512_variance_400.png:
0.11053592168574747

Posteriormente hay que hacer una segunda fase, para el ABJ hay que calcular la distancia euclideana para los primeros 8 parches para el de algoritmoBJTCD hay que hacer rw=5 de los 8 primeros.

# Algoritmo de Búsqueda Jerárquica

In [51]:
def algoritmoBusquedaJerarquica(posicionAltoImagenActual,posicionAnchoImagenActual,imgAct,imgRuido,areaBus,tamParche):   
#     imgActBlur = cv2.GaussianBlur(imgAct,(5,5),0)  
    imagenActual = imgAct[posicionAltoImagenActual:posicionAltoImagenActual+tamParche, posicionAnchoImagenActual: posicionAnchoImagenActual+tamParche]
    imagenActualBlur = cv2.GaussianBlur(imagenActual,(3,3),0) # Esta parte se inserta dado que el artículo dice que se agrega un filtro suavizado

    dh1 = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for m in range(posicionAltoImagenActual,posicionAltoImagenActual+areaBus-tamParche+1):
        for n in range(posicionAnchoImagenActual,posicionAnchoImagenActual+areaBus-tamParche+1):
            imagenReferencia = imgRuido[m:m+tamParche, n: n+tamParche]
            imagenReferenciaBlur = cv2.GaussianBlur(imagenReferencia,(3,3),0)
            lenParche=int(tamParche/2)
            ssd=0
            for rx_Pache in range (0,lenParche):
                for ry_Pache in range (0,lenParche):
                    rx_Parche_int = int(2*(rx_Pache))
                    ry_Parche_int = int(2*(ry_Pache))
                    resta = int(imagenActualBlur[rx_Parche_int,ry_Parche_int])-int(imagenReferenciaBlur[rx_Parche_int,ry_Parche_int])
                    ssd += resta**2                  
#                     ssd += np.sum(np.power((imagenActualBlur[rx_Parche_int,ry_Parche_int]-imagenReferenciaBlur[rx_Parche_int,ry_Parche_int]),2))
            dh1 = dh1.append({'ssd': ssd, 'coordenada_x':m, 'coordenada_y':n}, ignore_index=True)
#     print(posicionAltoImagenActual,posicionAnchoImagenActual)
    by_ascending_ssd = dh1.sort_values('ssd',ascending=True)
    by_ascending_ssd = by_ascending_ssd.reset_index(drop=True)
    #by_ssd.head()
#     print(by_ascending_ssd)
    
    ##Aquí comienza la segunda etapa
    dhFefinitive = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for iteraLista in range(0,8):
        Alto_x = by_ascending_ssd['coordenada_x'][iteraLista]
        Alto_y = by_ascending_ssd['coordenada_y'][iteraLista]
        imagenReferenciaDos = imgRuido[Alto_x:Alto_x+tamParche, Alto_y: Alto_y+tamParche]
        imagenReferenciaDosBlur = cv2.GaussianBlur(imagenReferenciaDos,(3,3),0)
        ssd=0
        for i in range (0,tamParche):
            for j in range (0,tamParche):
                resta = int(imagenActualBlur[i,j])-int(imagenReferenciaDosBlur[i,j])
                ssd += resta**2                  
#                 ssd += np.sum(np.power((imagenActualBlur[i,j]-imagenReferenciaDosBlur[i,j]),2))
        dhFefinitive = dhFefinitive.append({'ssd': ssd, 'coordenada_x':Alto_x, 'coordenada_y':Alto_y}, ignore_index=True)
    by_ascending_dhFefinitive = dhFefinitive.sort_values('ssd', ascending=True)
    by_ascending_dhFefinitive = by_ascending_dhFefinitive.reset_index(drop=True)
#     print(by_ascending_dhFefinitive)
    posicionAltoImagenReferencia = by_ascending_dhFefinitive['coordenada_x'][0]
    posicionAnchoImagenReferencia = by_ascending_dhFefinitive['coordenada_y'][0]
    
    return posicionAltoImagenReferencia,posicionAnchoImagenReferencia

In [68]:
pxy_pref = pd.DataFrame(columns=['coordenada_x', 'coordenada_y','referencia_x','referencia_y',])
altoImagen = imagenReferenciaGris.shape[0]
anchoImagen = imagenReferenciaGris.shape[1]

for posicionAlto in range(400,altoImagen-areaBusqueda): # altoImagen-areaBusqueda
    for posicionAncho in range(400,anchoImagen-areaBusqueda): # anchoImagen-areaBusqueda
        posicionAltoImagenReferencia,posicionAnchoImagenReferencia = algoritmoBusquedaJerarquica(posicionAlto,posicionAncho,imagenReferenciaGris,imagenRuidoGris,areaBusqueda,tamañoParche)
        pxy_pref = pxy_pref.append({'coordenada_x': posicionAlto, 'coordenada_y':posicionAncho, 'referencia_x':posicionAltoImagenReferencia,'referencia_y':posicionAnchoImagenReferencia}, ignore_index=True)
    
    
        
pxy_pref
# plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
# plt.show()
#print(imagen)

,coordenada_x,coordenada_y,referencia_x,referencia_y
0,400,400,402,413
1,400,401,402,414
2,400,402,401,413
3,400,403,400,412
4,400,404,400,413
...,...,...,...,...
8276,490,486,490,488
8277,490,487,490,489
8278,490,488,490,490
8279,490,489,490,491


In [69]:
suma = 0
miu = 0
for i in range(0,len(pxy_pref)-1):
    if pxy_pref['coordenada_x'][i] == pxy_pref['referencia_x'][i] and pxy_pref['coordenada_y'][i] == pxy_pref['referencia_y'][i]:
        suma += 1

miu =  suma/(len(pxy_pref)-1)
miu
    

0.2579710144927536

lena_gray_512_512_variance_25_Efectivo.png

Con la TDC en la imágen pequeña r²
0.2927--------> Este hay que sacarlo completo

lena_gray_512_512_variance_100_Efectivo.png

Con la TDC en la imágen pequeña r²
0.2815289530446325

Cuando K = 20 y r = 8
# 1

Cuando K = 21 y r = 8
# 1

lenna_gaussiano_25 Cuando K = 21 y r = 8
# 1

# Transformada de coseno discreta

In [3]:
#imagenGrises = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)
def transformadaCosenoDiscreta(imagenGrises):
    imgFlotante= imagenGrises.astype('float')
    C_temp = np.zeros(imagenGrises.shape)
    dst = np.zeros(imagenGrises.shape)

    m = imagenGrises.shape[0]
    n = imagenGrises.shape[1]

    N = n
    C_temp[0, :] = 1 * np.sqrt(1/N)

    for indiceAltura in range (1,m):
        for indiceAncho in range(n):
            C_temp[indiceAltura, indiceAncho] = np.cos(np.pi * indiceAltura * (2*indiceAncho+1) / (2 * N ))* np.sqrt(2 / N )


    dst = np.dot(C_temp, imgFlotante)
    dst = np.dot(dst, np.transpose(C_temp))
    # dst1 = np.log(abs(dst))  #Valor absoluto
    return dst


# Algoritmo de Búsqueda Jerárquica en el dominio de la Transformada de coseno discreta

In [4]:
def algoritmoBJTCD(posicionAltoImagenActual,posicionAnchoImagenActual,imgAct,imgRuido,areaBus,tamParche):
#     imagenActualFiltro = transformadaCosenoDiscreta(imgAct)
#     imagenRuidoFiltro = transformadaCosenoDiscreta(imgRuido)

    imagenActual = imgAct[posicionAltoImagenActual:posicionAltoImagenActual+tamParche, posicionAnchoImagenActual: posicionAnchoImagenActual+tamParche]
    imagenActualTCD = transformadaCosenoDiscreta(imagenActual)
    dh1 = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for m in range(posicionAltoImagenActual,posicionAltoImagenActual+areaBus-tamParche+1):
        for n in range(posicionAnchoImagenActual,posicionAnchoImagenActual+areaBus-tamParche+1):
            imagenReferencia = imgRuido[m:m+tamParche, n: n+tamParche]
            imagenReferenciaTCD = transformadaCosenoDiscreta(imagenReferencia)
            lenParche=4 
            ssd=0
            for rx_Pache in range (0,lenParche):
                for ry_Pache in range (0,lenParche):
                    resta = int(imagenActualTCD[rx_Pache,ry_Pache])-int(imagenReferenciaTCD[rx_Pache,ry_Pache])
                    ssd += resta**2
#                     ssd += np.sum(np.power((imagenActualTCD[rx_Pache,ry_Pache]-imagenReferenciaTCD[rx_Pache,ry_Pache]),2))
            dh1 = dh1.append({'ssd': ssd, 'coordenada_x':m, 'coordenada_y':n}, ignore_index=True)
#     print(posicionAltoImagenActual,posicionAnchoImagenActual)
    by_ascending_ssd = dh1.sort_values('ssd',ascending=True)
    by_ascending_ssd = by_ascending_ssd.reset_index(drop=True)
    by_ascending_ssd = by_ascending_ssd.astype(int)
    #by_ssd.head()
#     print(by_ascending_ssd)
    
    ##Aquí comienza la segunda etapa
    dhFefinitive = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for iteraLista in range(0,8):
        Alto_x = by_ascending_ssd['coordenada_x'][iteraLista]
        Alto_y = by_ascending_ssd['coordenada_y'][iteraLista]
        imagenReferenciaDos = imgRuido[Alto_x:Alto_x+tamParche, Alto_y: Alto_y+tamParche]
        imagenReferenciaDosTCD = transformadaCosenoDiscreta(imagenReferenciaDos)
        ssd=0
        lenParche = 5
        for i in range (0,lenParche):
            for j in range (0,lenParche):
                resta = int(imagenActualTCD[i,j])-int(imagenReferenciaDosTCD[i,j])
                ssd += resta**2
#                 ssd += np.sum(np.power((imagenActualTCD[i,j]-imagenReferenciaDosTCD[i,j]),2))
        dhFefinitive = dhFefinitive.append({'ssd': ssd, 'coordenada_x':Alto_x, 'coordenada_y':Alto_y}, ignore_index=True)
    by_ascending_dhFefinitive = dhFefinitive.sort_values('ssd', ascending=True)
    by_ascending_dhFefinitive = by_ascending_dhFefinitive.reset_index(drop=True)
    by_ascending_dhFefinitive = by_ascending_dhFefinitive.astype(int)
#     print(by_ascending_dhFefinitive)
    posicionAltoImagenReferencia = by_ascending_dhFefinitive['coordenada_x'][0]
    posicionAnchoImagenReferencia = by_ascending_dhFefinitive['coordenada_y'][0]
    
    return posicionAltoImagenReferencia,posicionAnchoImagenReferencia


In [5]:
pxy_pref_BJTCD = pd.DataFrame(columns=['coordenada_x', 'coordenada_y','referencia_x','referencia_y',])
altoImagen = imagenReferenciaGris.shape[0]
anchoImagen = imagenReferenciaGris.shape[1]

for posicionAlto in range(0,altoImagen-areaBusqueda): # altoImagen-areaBusqueda
    for posicionAncho in range(250,anchoImagen-areaBusqueda): # anchoImagen-areaBusqueda
        posicionAltoImagenReferencia,posicionAnchoImagenReferencia = algoritmoBJTCD(posicionAlto,posicionAncho,imagenReferenciaGris,imagenRuidoGris,areaBusqueda,tamañoParche)
        pxy_pref_BJTCD = pxy_pref_BJTCD.append({'coordenada_x': posicionAlto, 'coordenada_y':posicionAncho, 'referencia_x':posicionAltoImagenReferencia,'referencia_y':posicionAnchoImagenReferencia}, ignore_index=True)
    
    
pxy_pref_BJTCD
# imagen = algoritmoBJTCD(imagenActual,areaBusqueda,tamañoParche)


,coordenada_x,coordenada_y,referencia_x,referencia_y
0,0,250,0,250
1,0,251,0,251
2,0,252,0,252
3,0,253,0,253
4,0,254,0,254
...,...,...,...,...
118326,490,486,500,491
118327,490,487,500,491
118328,490,488,500,494
118329,490,489,500,494


In [6]:
suma = 0
miu = 0
for i in range(0,len(pxy_pref_BJTCD)-1):
    if pxy_pref_BJTCD['coordenada_x'][i] == pxy_pref_BJTCD['referencia_x'][i] and pxy_pref_BJTCD['coordenada_y'][i] == pxy_pref_BJTCD['referencia_y'][i]:
        suma += 1

miu =  suma/(len(pxy_pref_BJTCD)-1)
miu
    

0.47706414265190566

Efectivo/lena_gray_512_512_variance_25_Efectivo.png:
Con la TDC en la imágen grande
0.7935332669653227
Con la TDC en la imágen pequeña r²
0.352166686939473
Efectivo/lena_gray_512_512_variance_100_Efectivo.png:
0.331820075194577


# Realización de otro commit veaamos

lena/lena_gray_512_512_variance_25.png:
0.286126414005445

lena/lena_gray_512_512_variance_100.png:
0.247335940669742

lena/lena_gray_512_512_variance_225.png:
0.17008941046029

lena/lena_gray_512_512_variance_400.png:
0.108866822122107

********Vamos por partes
(0,100)---->0.100755616203996
(100,200)---->0.31308173282551577
(200,300)---->0.1796166928043341
(300,400)---->0.1365811930996558
(400,anchoImagen-areaBusqueda)---->